In [ ]:
from skimage.io import imread
import numpy as np
from scipy.io import loadmat
from matplotlib import pyplot as plt
import time


In [ ]:
rootfolder = '.'

Load the image and rescale it in $[0,1]$

In [ ]:
img = imread(f'{rootfolder}/data/cameraman.png') / 255
img = img[50:200, 50:200]
imsz = img.shape

# patch size
p = 7

# number of elements in the patch
M = p ** 2

# radius of the search neighborhood
H = 12


Corrupt the image with white gaussian noise

In [ ]:
sigma_noise = 20/255
# noisy_img =

Compute the PSNR of the noisy input

In [ ]:
# psnr_noisy =


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
ax[0].imshow(img, cmap='gray')
ax[0].set_title('Original image')

ax[1].imshow(noisy_img, cmap='gray')
ax[1].set_title(f'Noisy image, PSNR = {psnr_noisy:.2f}')


Pixel-based NL-Means
--------------------
Set parameters and initialize the variables

In [ ]:
# initialize the estimated image
img_hat = np.zeros_like(img)

# pad the noisy image
# img_pad =


Main loop

In [ ]:
for i in range(0, imsz[0]):
    for j in range(0, imsz[1]):
        # extrach the patch from img_pad whose center has the location (i, j) in the noisy image
        # s =

        # initialize the pixel estimate and the total weights
        pixel_hat = 0
        weight = 0

        # extract all the patches in the search neighborhood
#        for r
#            for c
                # extract the patch
                # z =

                # compute the distance with the reference patch
                # d =

                # update the weight and the pixel estimate
                # pixel_hat
                # weight

        # estimate the pixel (i, j) as the weighted average of the central pixel of the extracted patches
        # img_hat[i, j] =


Compute the PSNR of the estimated image

In [ ]:
# psnr_hat =
plt.figure(figsize=(10,10))
plt.imshow(img_hat, cmap='gray')
plt.title(f'Estimated Image,\nPSNR = {psnr_hat:.2f}')
